In [14]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# loading necessary dependies 
import pandas as pd
from collections import  Counter
import re
import numpy as np
import math
import random


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

from gensim.parsing.preprocessing import remove_stopwords

import spacy
sp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
"""
Function to convert .data to pandas
"""
def file_dataframe(filename):
  input_string = '/content/drive/Shareddrives/Text Analytics/HW2/' + str(filename) + '.data'
  open_file = open(input_string, 'r')
  read_data = open_file.readlines()

  df = pd.DataFrame(read_data)

  # convert to dataFrame

  out = pd.DataFrame(df[0].str.split('|',2).tolist(),columns=['Word','Sense','Text'])

  return out


In [17]:
train_df = file_dataframe('train')
val_df = file_dataframe('validate')
test_df = file_dataframe('test')

In [18]:
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 
from nltk.tokenize import word_tokenize 

# from nltk.stem import WordNetLemmatizer 
  
# lemmatizer = WordNetLemmatizer() 

def clean_text(text):
  filtered_sentence = remove_stopwords(text.lower())

  res = re.sub(r'[^\w\s]', '', filtered_sentence)
  # words = word_tokenize(res)
  lem = sp(res)
  return ' '.join([w.lemma_ for w in lem if len(w) > 2]) # for words greater than 2 { example : s, dr, 16}

In [19]:
def get_words(text,n):
  search_text = ' '.join(text.split()[1:])
  keyword = text.split()[0]
  before_keyword, keyword, after_keyword = search_text.partition(keyword)
  feature_words = ', '.join(after_keyword.split()[:n] + before_keyword.split()[-n:])

  return feature_words

In [20]:
def main_function(train_df, val_df, test_df, lam, num_of_features):

  val_df['Word'] = val_df['Word'].str.strip()
  # val_df.head(5)

  train_df['Word'] = train_df['Word'].str.strip()
  # train_df.head(5)
  
  # getting prior probabilities
  prior_count = train_df.groupby(['Word','Sense']).size().reset_index().rename(columns={0:'count'}) # get c ( word | sense)
  # prior_count.head()
  # get word counts 
  word_count = train_df.groupby(['Word']).size().reset_index().rename(columns={0:'total'}) # get c(word)

  # join / merge 
  prior_prob = prior_count.merge(word_count, how='left')

  prior_prob['prob'] = prior_prob['count'] / prior_prob['total'] # get p(word|sense) = c ( word | sense) / c(word)

  prior_prob = prior_prob[['Word','Sense','count','prob']]

  prior_prob['word,sense'] = prior_prob['Word'] + prior_prob['Sense']

  # saving prior probabilities into a  dict
  word_prior = pd.Series(prior_prob['prob'].values, index=tuple(prior_prob['word,sense'])).to_dict()

  # # get words - 
  # temp_df = prior_count
  # words = list(temp_df['Word'].unique())

  # base_sense = []

  # for e in words:
  #   most_sense = temp_df[temp_df['Word']==e]

  #   most_count = most_sense['count'].max()

  #   most_sense_baseline = most_sense[most_sense['count']==most_count]['Sense'].tolist()

  #   base_sense.append(tuple([e, most_sense_baseline[0]]))

  # sense_dict = {}
  # for am_sense in base_sense:
  #   sense_dict[am_sense[0].strip()] = am_sense[1]

  # # baseline accuracy 
  # for ind in val_df.index:
  #   val_df['pred'] = ''
  #   disamb_word = val_df['Word'][ind]

  #   val_df['pred'] = sense_dict[disamb_word]

  # # converting to int 
  # val_df['Sense'] =pd.to_numeric(val_df['Sense'])
  # val_df['pred'] =pd.to_numeric(val_df['pred'])
  # val_df['match'] = np.where((val_df['Sense'] - val_df['pred'])==0,'Right', 'Wrong')
  
  # print('Baseline Validation accuracy is: ', val_df.groupby('match').pred.count()[0]/val_df.shape[0])

  x = train_df['Word'].unique().tolist()

  try_df = prior_count

  df = []
  for idx, i in enumerate(x):
    try_temp = try_df.loc[try_df['Word']==i]
    v = try_temp['count'].max()
    df.append(tuple([i,try_temp.loc[try_temp['count']==v]['Sense'].tolist()[0]]))

  new_df = pd.DataFrame.from_records(df, columns =['Word', 'pred'])
  # new_df.head(5)

  val_df['Word'] = val_df['Word'].str.strip()
  # val_df.head(5)

  train_df['Word'] = train_df['Word'].str.strip()
  # train_df.head(5)
  test_df1=val_df.merge(new_df,how='left',on='Word')

  a = test_df1['Sense'].tolist()
  a = [int(i) for i in a]
  b = test_df1['pred'].tolist()
  b = [int(i) for i in b]

  from sklearn.metrics import confusion_matrix
  cm_simple=confusion_matrix(a,b)
  # cm_simple
  accuracy = cm_simple.diagonal().sum()/sum(cm_simple).sum()*100

  print('baseline accuracy -> ', accuracy)

  df = train_df
  df['word_am'] = df['Word'].apply(lambda x : x.split('.')[0])
  df['clean_text']= df['Text'].apply(lambda x: clean_text(x))
  df['temp'] =  df['word_am'] + ' '+ df['clean_text']

  # get vocab length 
  vocab = df['clean_text'].str.cat()
  vocab_length = len(set(word_tokenize(vocab)))

  df['feature'] = df['temp'].apply(lambda x : get_words(x,num_of_features))

  feature_df = df[['Word','Sense','feature']] # filter out essesntial columns 

  feature_df['word_sense'] = feature_df['Word'] + feature_df['Sense'] # combining in %format -> "word.pos sense"

  merge_df =  feature_df.groupby('word_sense')['feature'].apply(' '.join).reset_index() #grouping words in the same format of word.pos sense

  merge_df['count'] = merge_df['feature'].apply(lambda x : Counter(x.split(','))) # counting word and their occurense in the "word.pos sense"

  prior_count['word_sense'] = prior_count['Word'] + prior_count['Sense'] # gets word_sense count for getting p(f|sense) = c(f|sense) / c(word,sense)

  sense_count = prior_count[['word_sense','count']]

  feature_sense_df = sense_count.merge(merge_df, how='left', on='word_sense')

  # add -1 lambda smoothing to get the probabilities of the (feature|sense)
  for ind in feature_sense_df.index:
    val_dict = feature_sense_df['count_y'][ind]
    tot = feature_sense_df['count_x'][ind]
    for k,v in val_dict.items():
      val_dict[k] = (v+lam)/(tot + (lam*vocab_length)) # add-1 lambda smoothing 


  final_featureSense = feature_sense_df[['word_sense','count_y']]

  feature_probs= pd.Series(final_featureSense['count_y'].values, index=tuple(final_featureSense['word_sense'])).to_dict()

  val_df['clean_text'] = val_df['Text'].apply(lambda x: clean_text(x))
  val_df['temp'] = val_df['Word'].apply(lambda x : x.split('.')[0]) + ' ' + val_df['clean_text']
  val_df['temp'] = val_df['temp'].apply(lambda x : get_words(x,num_of_features))


  val_df['pred'] = ''
  for ind in val_df.index:
    r = val_df['Word'][ind].split()[0]
    w = val_df['temp'][ind].split(',')
    li = []
    sense_li = []
    final_list = []
    prob_dict = {}
    for element in list(feature_probs.keys()):
      if str(r) == element.split()[0]:
        li.append(element)
      
      for sense in li:
        for we in w:
          if we in list(feature_probs[sense].keys()):
            prob_dict[we] = feature_probs[sense][we]
          else:
            prob_dict[we] = lam/(lam*vocab_length)
      
        prob_mul_add = math.log(word_prior[sense])
        for v in prob_dict.values():
          prob_mul_add += math.log(v)

        final_list.append(tuple([math.exp(prob_mul_add),sense]))

    val_df['pred'][ind] = sorted(final_list)[0][1].split()[1]

    # getting validation accuracy 
  a = val_df['Sense'].tolist()
  a = [int(i) for i in a]
  b = val_df['pred'].tolist()
  b = [int(i) for i in b]

  from sklearn.metrics import confusion_matrix
  cm_simple=confusion_matrix(a,b)
  # cm_simple
  accuracy = cm_simple.diagonal().sum()/sum(cm_simple).sum()*100

  print('For features : ' + str(num_of_features) + '  and lambda: '  + str(lam))

  print('Validation accuracy is: '+ str(accuracy) )

  return word_prior, vocab_length, feature_probs

Checking value

In [21]:
# word_prior, vocab_length, feature_probs = main_function(train_df, val_df, test_df, 1,1)

for different lambda and different num of features 

In [ ]:
for lam in [1,0.1,0.0001]:
  for num_of_features in [1,3,7]:
    word_prior, vocab_length, feature_probs = main_function(train_df, val_df, test_df, lam,num_of_features)

baseline accuracy ->  81.13612004287245


# working on test data saving it. 

In [ ]:
def save_test(test_df, word_prior, vocab_length, feature_probs, best_lam, best_num_of_features):
  # clean_text 
  test_df['clean_text'] = test_df['Text'].apply(lambda x: clean_text(x))
  test_df['temp'] = test_df['Word'].apply(lambda x : x.split('.')[0]) + ' ' + test_df['clean_text']
  test_df['temp'] = test_df['temp'].apply(lambda x : get_words(x,best_num_of_features))

  test_df['pred'] = ''
  for ind in test_df.index:
    r = test_df['Word'][ind].split()[0]
    w = test_df['temp'][ind].split(',')
    li = []
    sense_li = []
    final_list = []
    prob_dict = {}
    for element in list(feature_probs.keys()):
      if str(r) == element.split()[0]:
        li.append(element)
      
      for sense in li:
        for we in w:
          if we in list(feature_probs[sense].keys()):
            prob_dict[we] = feature_probs[sense][we]
          else:
            prob_dict[we] = best_lam/ (best_lam * vocab_length)
      
        prob_mul_add = 0
        for v in prob_dict.values():
          prob_mul_add += math.log(v)

        final_list.append(tuple([math.exp(prob_mul_add),sense]))

    test_df['pred'][ind] = sorted(final_list)[0][1].split()[1]

    final_data = test_df[['Word','pred','Text']]

  text_file = open("WSD-test.data", "w")
  for ind in final_data.index:
    line = final_data['Word'][ind] + ' | ' + final_data['pred'][ind]+ ' | ' + final_data['Text'][ind] 
    text_file.write(line)

  text_file.close()

  print('done')

In [ ]:
best_lam = 0.1
best_num_of_features = 3
save_test(test_df, word_prior, vocab_length, feature_probs, best_lam, best_num_of_features)

In [ ]:
input_string = '/content/sample.data'
open_file = open(input_string, 'r')
read_data = open_file.readlines()

df = pd.DataFrame(read_data)

# convert to dataFrame

out = pd.DataFrame(df[0].str.split('|',2).tolist(),columns=['Word','Sense','Text'])